## Intro

This workshop will explain some extended reasoning behind information entropy and Kullback-Leibler divergence. In short, ability to accurately predict symbols allows for more efficient compression of information. This efficiency of compression is often used as a proxy to predictive model quality if the form of LogLikelihood loss.

##### Acknowledgements
The RNN parts were heavily inspired by [karpathy/char-rnn](https://github.com/karpathy/char-rnn)

But first what is Kullback-Leibler divergence, and what is entropy.

### Kullback-Leibler divergence

$D_{KL}(P || Q) = \sum_{x\in X} P(x)  log \frac{P(x)}{Q(x)}$

This can be viewed as expectation over log of ratio $P$ and $Q$ . Do note that things get weird if $P(x)$ or $Q(x)$ could be zero, so just pretend that its $\epsilon > 0$ in that case. There will be some explanations for such cases later.

So, $E_{P(x)} [\log \frac{P(x)}{Q(x)}]$, for some symbol $x$ that is sampled from distribution $P$, what would be the extra amount of bits needed to encode it if we created our encoding table for distribtution $Q$.
Ops, bits, encoding table, we did not say anything about that yet, so, lets put a bookmark here $Bookmark 1.$ and we will continue this after we dealt with encoding.


### Encoding

Encoding is all about replacing $x\in X$ with sequences $y_1,y_2,\dots,y_k$ where $y_i\in Y$, such that those sequences can be transformed back into original symbols.

Specifically, what we care about most is replacement with binary sequences $\{1, 0\}_k$ i.e. bits. The end goal is to use as little bits as needed to encode a sequence written in symbols being encoded. So as not to get entagled in theory too much, we will jump straight to a method of getting such codes, [Huffman coding](https://en.wikipedia.org/wiki/Huffman_coding).

In [1]:
import heapq

def huffman(symbol_freq_pairs):
    heap = [[freq, (sym, freq, [])] for sym, freq in symbol_freq_pairs]
    heapq.heapify(heap)
    while len(heap) > 1:
        a = heapq.heappop(heap)
        b = heapq.heappop(heap)
        for sym, freq, code in a[1:]:
            code.append('0')
        for sym, freq, code in b[1:]:
            code.append('1')
        heapq.heappush(heap, [a[0] + b[0]] + a[1:] + b[1:])
    return {sym:(freq, ''.join(code[::-1])) for sym, freq, code in heapq.heappop(heap)[1:]}

def pretty(coding_table, sort_key=lambda x: (len(x[1][1]), x[0])):
    for sym, (freq, code) in sorted(coding_table.items(), key=sort_key):
        print("{:3} {:5.4f} {:10} {}".format(sym, freq, code, len(code)))

Lets see an example for this, first uniform distribution, then for a random distribution.

In [2]:
import numpy as np
import random
from collections import Counter
np.random.seed(42)
random.seed(42)

symbols = [chr(i) for i in range(ord('a'), ord('z') + 1)]
uniform = zip(symbols, np.full(len(symbols), 1.0 / len(symbols)))
print("Codes for uniform distribution.")
pretty(huffman(uniform))

N = 50
random_count = Counter(np.random.choice(symbols, N))
random = zip(symbols, [random_count[sym] / N for sym in symbols])
print("Codes for random distribution.")
pretty(huffman(random))

Codes for uniform distribution.
u   0.0385 0000       4
v   0.0385 0001       4
w   0.0385 0010       4
x   0.0385 0011       4
y   0.0385 0100       4
z   0.0385 0101       4
a   0.0385 01100      5
b   0.0385 01101      5
c   0.0385 01110      5
d   0.0385 01111      5
e   0.0385 10000      5
f   0.0385 10001      5
g   0.0385 10010      5
h   0.0385 10011      5
i   0.0385 10100      5
j   0.0385 10101      5
k   0.0385 10110      5
l   0.0385 10111      5
m   0.0385 11000      5
n   0.0385 11001      5
o   0.0385 11010      5
p   0.0385 11011      5
q   0.0385 11100      5
r   0.0385 11101      5
s   0.0385 11110      5
t   0.0385 11111      5
Codes for random distribution.
u   0.1000 001        3
g   0.0800 1100       4
k   0.0600 0101       4
l   0.0800 1110       4
o   0.0600 0110       4
s   0.0600 1000       4
w   0.0400 0000       4
x   0.0600 1001       4
y   0.0400 0001       4
z   0.0400 0100       4
b   0.0400 01111      5
c   0.0400 10100      5
h   0.0400 10111      5
r

Now, Huffman coding operates per-symbol basis, so there is some inefficiencies (i.e. if there are only 2 symbols to begin with, they will be replaced with 1 and 0 regardless of their frequencies). It behaves optimally if all frequencies are powers of 2.

In [3]:
import math
powers2 = [('a', 1/4),
           ('b', 1/8),
           ('c', 1/16),
           ('d', 1/2),
           ('e', 1/16)
          ]
pretty(huffman(powers2), sort_key=lambda x: x[0])

print("Note that:")
for f in [x[1] for x in powers2]:
    print("-log2({}) = {}".format(f, -math.log2(f)))

a   0.2500 00         2
b   0.1250 010        3
c   0.0625 0110       4
d   0.5000 1          1
e   0.0625 0111       4
Note that:
-log2(0.25) = 2.0
-log2(0.125) = 3.0
-log2(0.0625) = 4.0
-log2(0.5) = 1.0
-log2(0.0625) = 4.0


The theoretical minimum of bits-per-symbol coding is $-log_2(symbol\_frequency)$ if the expected number of bits per symbol wants to be minimized $E_{P(x)} [- log_2 P(x)]$. (Proof omitted)

This expectation in extended form is $- \sum_{x\in X} P(x) log_2 P(x)$ (hey, that looks like entropy)

But what if we make the coding using the "wrong" wrong distribution, say $Q$.

In [4]:
wrong_powers2 = [('a', 1/16),
                 ('b', 1/4),
                 ('c', 1/2),
                 ('d', 1/16),
                 ('e', 1/8)
                ]
print("Good code")
pretty(huffman(powers2), sort_key=lambda x: x[0])
print("Wrong code")
pretty(huffman(wrong_powers2), sort_key=lambda x: x[0])

print("Note that:")
for f1, f2 in zip([x[1] for x in powers2], [x[1] for x in wrong_powers2]):
    print("log2({} / {}) = {}".format(f1, f2, math.log2(f1/f2)))

Good code
a   0.2500 00         2
b   0.1250 010        3
c   0.0625 0110       4
d   0.5000 1          1
e   0.0625 0111       4
Wrong code
a   0.0625 0000       4
b   0.2500 01         2
c   0.5000 1          1
d   0.0625 0001       4
e   0.1250 001        3
Note that:
log2(0.25 / 0.0625) = 2.0
log2(0.125 / 0.25) = -1.0
log2(0.0625 / 0.5) = -3.0
log2(0.5 / 0.0625) = 3.0
log2(0.0625 / 0.125) = -1.0


The negative $log_2$ of symbol probability ratio is the difference in number of bits needed to represent those symbols, and the expectation of that over $P(x)$ is the expected number of extra bits $E_{P(x)} [\log \frac{P(x)}{Q(x)}]$ (hey this was $Bookmark1$ we are back on track).

Also, $log_2 \frac{P(x)}{Q(x)} = - log_2 Q(x) - (- log_2P(x))$, so there that difference becomes obvious.

But lets get back to $D_{KL}(P || Q) = \sum_{x\in X} P(x)  log \frac{P(x)}{Q(x)}$.

We can split that to $\sum_{x\in X} P(x)  log \frac{P(x)}{Q(x)} = -\sum_{x\in X} P(x) log_2 Q(x) - (- \sum_{x\in X} P(x) log_2 P(x))$, which is the difference between expected bits per symbol when using coding table constructed for $Q(x)$ and when using one constructed for $P(x)$ which would be the optimal code. The second component is the _Shannon entropy_ and is the minimum of bits needed to encode data under distribution $P(x)$.

Hey, $-\sum_{x\in X} P(x) log_2 Q(x)$ is cross-entropy and also that negative LogLikelihood we love to optimize!

A very, very important thing to understand is: _What is $P(x)$?_ But for that, lets get some data first.

### Shakespearian datasets

We are gonna grab Shakespeares collected works from https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt.

In [5]:
import urllib.request
import os

if not os.path.isfile('shakespeare.txt'):
    shakespeare_by_karpathy = 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt'
    urllib.request.urlretrieve(shakespeare_by_karpathy, 'shakespeare.txt')

In [6]:
shakespeare_raw = open('shakespeare.txt', 'r').readlines()
shakespeare = []
shake_it = iter(shakespeare_raw)
speaker = next(shake_it).rstrip()[:-1]
lines = []
while True:
    try:
        line = next(shake_it).rstrip()
        if line:
            lines.append(line)
        else:
            shakespeare.append(' '.join(lines))
            lines = []
            speaker = next(shake_it).rstrip()[:-1]
    except StopIteration:
        shakespeare.append(''.join(lines))
        del lines
        break

In [7]:
len(shakespeare), shakespeare[:10]

(31023,
 ['Before we proceed any further, hear me speak.',
  'Speak, speak.',
  'You are all resolved rather to die than to famish?',
  'Resolved. resolved.',
  'First, you know Caius Marcius is chief enemy to the people.',
  "We know't, we know't.",
  "Let us kill him, and we'll have corn at our own price. Is't a verdict?",
  "No more talking on't; let it be done: away, away!",
  'One word, good citizens.',
  'We are accounted poor citizens, the patricians good. What authority surfeits on would relieve us: if they would yield us but the superfluity, while it were wholesome, we might guess they relieved us humanely; but they think we are too dear: the leanness that afflicts us, the object of our misery, is as an inventory to particularise their abundance; our sufferance is a gain to them Let us revenge this with our pikes, ere we become rakes: for the gods know I speak this in hunger for bread, not in thirst for revenge.'])

So, our samples are lines by speakers, sort of. I will make an important distiction now, we shall have 2 datasets, one will be _The Complete Shakespeare_ noted with $P_{CS}$ and _Shakespeare Lost Works_ noted with $P_{LW}$.

The difference is that $P_{CS}$ is fully known, since it is defined by the data above, $P_{LW}$ implies some lost works by Shakespeare, so the dataset above is just a sample from the full distribution.

Example:

In [8]:
let_us = [x for x in shakespeare if x.startswith('Let us ')]
len(let_us), let_us

(29,
 ["Let us kill him, and we'll have corn at our own price. Is't a verdict?",
  'Let us take the law of our sides; let them begin.',
  'Let us entreat you stay till after dinner.',
  "Let us to the great supper: their cheer is the greater that I am subdued. Would the cook were of my mind! Shall we go prove what's to be done?",
  "Let us withdraw; 'twill be a storm.",
  'Let us deal justly. Sleepest or wakest thou, jolly shepherd? Thy sheep be in the corn; And for one blast of thy minikin mouth, Thy sheep shall take no harm. Pur! the cat is gray.',
  "Let us go. Come; Our separation so abides, and flies, That thou, residing here, go'st yet with me, And I, hence fleeting, here remain with thee. Away!",
  'Let us go. Good Enobarbus, make yourself my guest Whilst you abide here.',
  "Let us score their backs, And snatch 'em up, as we take hares, behind: 'Tis sport to maul a runner.",
  'Let us sit and mock the good housewife Fortune from her wheel, that her gifts may henceforth be besto

In [9]:
Counter(x.split()[2] for x in let_us)

Counter({'kill': 1,
         'take': 1,
         'entreat': 1,
         'to': 1,
         'withdraw;': 1,
         'deal': 1,
         'go.': 2,
         'score': 1,
         'sit': 1,
         'not': 1,
         'do': 1,
         'go': 1,
         'from': 2,
         'make': 2,
         'seek': 1,
         'rather': 1,
         'hear,': 1,
         'confess': 1,
         'haste': 1,
         'wag,': 1,
         'about': 1,
         'condole': 1,
         'on,': 1,
         'withdraw': 1,
         'first': 1,
         'leave': 1})

This means that $P_{CS}("Let\space us ") = 29 / 31023$ and $P_{CS}("make" | "Let\space us ") = 2 / 29$ since we know all the possible samples, it is not possible to do the same for $P_{LW}$ since there might be more cases in the unknown lost works.

On that note, observe that $P_{CS}("k" | "Let\space us ") = 1 / 29$ since there is only the one "kill" starting with 'k', but also $P_{CS}("i" | "Let\space us \space k") = 1.0$.

Since the prefix "Let us k" uniquely identifies the whole sample "Let us kill him, and we'll have corn at our own price. Is't a verdict?", if we were to encode that sequence, we would only need to encode "Let us k", as the rest is fully defined by $P_{CS}$. This is one of those edge cases mentioned near the beginning concerning $P(x) = 0.0$ issues, as in such cases we operate on $X' \subset X$ rather than $X$ which breaks the math a little, but is not an issue in practice (note above how Huffman code produced codes for symbols with $0.0$ probability).

The takeway for $P(x)$ is that in a lot of practical applications, you just don't know it, but rather you have some samples $s_i$ from it. With those samples you fit some $Q(x)$ so that the samples _could_ have come from it. The issue is that if you assume the samples are all there is and fully represent $P(x)$, $Q(x)$ could diverge significantly for $x'\in X$ you do not have in your samples, aka overfitting. The information theory wisdom here says that in that case you minimize combined size of produced code (prediction loss) and coding table size (regularization loss, or simply model complexity).

Huh, but _why_?

Lets recall $D_{KL}(P || Q) = -\sum_{x\in X} P(x) log_2 Q(x) - (- \sum_{x\in X} P(x) log_2 P(x))$
I.E. KL is cross-entropy minus entropy. We don't know $P(x)$, we have a bunch of samples from it though. Lets say $Q(x)$ would generate those samples as well.

##### Quick digression
Don't get confused by "generating" samples here in connection to classification models. In classification as sample is a $s_i = (x_i, y_i)$ pair, you have some other $G(x)$ that generates inputs for classificators, and then you have $P(y_i|x_i)$ and $Q(y_i|x_i)$ as stand-ins for joint distributions $P(x_i, y_i)$ and $Q(x_i, y_i)$. For things like images and other high-dimensonal input we rarely even near the entirety of $x\in X$ being sampled so $G(x)$ and therefore joint distributions are unknown. $P(y|x)$ is mostly set to $1.0$ for known samples and adjoined label, although there are some cases when it is not so (model distillation, or other uncertainty). It should be noted that this in itself is a mistake in the grand scheme of things, but we don't know any better than to set it to $1.0$ and roll with that.

All of the things said so far still stand, if you want to transmit a set of $(image, label)$ pairs, and you have a classifier $Q(label|image)$ that perfectly classifies images you can omit the labels from the coded sequence.

###### Mini-quick digression
If $x \in X$ are all possible images what would the distribution of natural images (well, photographs without digital distortions) $G(x)$ look like? Very minor modifications can make an image 'unnatural', a lot of adversial attacks exploit the fact that we don't know when we are given an image outside of the distribution our model should work for.

##### Quick digression end

Soooo, we were talking about making $Q(x)$ work on input not present in sampled dataset. Since $P(x)$ is what it is, we can only make an impact in $-\sum_{x\in X} P(x) log_2 Q(x)$, wherein we assume that $\delta P(x)$ behaves similar in scale to $\delta x$ in the vicinity of $x$, in other words $P(x)$ is locally smooth, and simpler models are more locally smooth than complicated ones. This is in the end just hopefull thinking, but it works out well, see also natural gradients for a slightly more elaborate approach to this. Once again, refer to mini-quick digression above, $P(x)$ tends to be more "sparse" than we can make $Q(x)$, so when modelling $Q(x)$ we allow it to generate samples "in between", even though they would not come from the true distribution.

This is a bit of handwaving, but unless you are using probabilistic models and fitting assumed distributions to your data, you are in discriminative black magic territory, and can only mess around with smoothness by changing model complexity, adding regularization, averaging outputs, data augumentation and other, and none of these methods promise anything but often help.


### Entropy

What is entropy?
Most of the time you run into it in practice is while working with a sequence of bytes, and you get something like:
$entropy = - \sum_{i=0 \to 256} \frac{count(i)}{N} log_2 \frac{count(i)}{N}$, with minimum being 0 when there is only one byte value present, and 8 for equally frequent bytes.

Well, that is a dirty lie. Observe:

In [10]:
def byte_entropy(sequence):
    entropy = 0
    for sym, count in Counter(sequence).items():
        freq = count / len(sequence)
        entropy -= freq * math.log2(freq)
    return entropy

In [11]:
sequence = 'the quick brown fox jumps over the lazy dog'
print(sequence, byte_entropy(sequence))
shuffled = np.array([x for x in sequence])
np.random.shuffle(shuffled)
print(''.join(shuffled), byte_entropy(shuffled))

the quick brown fox jumps over the lazy dog 4.385453417442482
etytoxbjq  a enhmporrol uuz eogfdciv h kws  4.385453417442482


One of these is much more 'predictable' than the other, if I tell you "the quick brown fox" you can probably guess what the rest of should be __if our samples come from english language distribution__ (you need to know that beforehand). What did we miss?

Lets look at $- \sum_{x\in X} P(x) log_2 P(x)$

Hmm. $- \sum_{j=1 \to K} \sum_{x \in X} P(x) log_2 P(x)$ lets just look at the sum of needed bits to encode.

Uh, oh, wait a moment, we did not make use $j$ anywhere in there. Lets try again.

$- \sum_{j=1 \to K} \sum_{x \in X} P_j(x) log_2 P_j(x)$, better, but what in the world would be $P_j$, that makes even less sense.

$- \sum_{j=1 \to K} \sum_{x \in X} P(x|x_{j-1},\dots,x_1) log_2 P(x|x_{j-1},\dots,x_1)$ Better! But this is the number of bits needed to encode some specific sequence, what about a general estimate?

$- \sum_{(x_1, \dots, x_k), x \in X, k \in N} P(x_1,\dots,x_k) \sum_{j=1 \to K} \sum_{x \in X} P(x|x_{j-1},\dots,x_1) log_2 P(x|x_{j-1},\dots,x_1)$

What in the world is that? Well, its an expectation of bits needed to encode/decode a single character over all possible sequences with the code being allowed to know the already decoded characters (for obvious reasons it can't use not yet decoded characters, assuming you do get a sequence of encoded characters, rather then, say, an embedding which is a single information used to seed a decoder).

Now, unless we are working with the completed works of Shakespeare, we can't know $P(x_1,\dots,x_k)$, but given samples we have a pretty decent shot of finding a $Q(x|x_{j-1},\dots,x_1)$ that is similar to $P(x|x_{j-1},\dots,x_1)$.

Disclaimer: there are many ways to encode stuff, this is just a variant on Huffman code.

Thats it, thats the theory, finally done, lets train some models.

Encode symbols.

In [12]:
from collections import Counter
tag_counts = Counter('$'.join(shakespeare))
tag_map = {tag:i for i, tag in enumerate(sorted(tag_counts.keys()))}
tag_map['<start>'] = len(tag_map)
tag_map['<pad>'] = len(tag_map)
tags = [tag for tag, i in sorted(tag_map.items(), key=lambda x: x[1])]

Define the model. This is set up for batch size 1, I had issues getting it to run with padded sequences, but this works fine.

In [13]:
import torch
import torch.nn as nn
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# device = torch.device('cuda')
device = torch.device('cpu')

class ShakespeareRNN(nn.Module):
    def __init__(self, num_layers, hidden_size=100, embedding_dim=50):
        super(ShakespeareRNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        padding_idx = tag_map['<pad>']
        self.embedding = nn.Embedding(
            num_embeddings=len(tags),
            embedding_dim=embedding_dim,
            padding_idx=padding_idx
        )
        
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        
        self.initial_hidden = (nn.Parameter(torch.zeros(num_layers, 1, hidden_size).to(device=device)),
                               nn.Parameter(torch.zeros(num_layers, 1, hidden_size).to(device=device)))
        
        self.output = nn.Linear(hidden_size, len(tags)-2)
        
    def forward(self, X, X_lenghts, hidden_state=None):        
        X = self.embedding(X)
        # X = torch.nn.utils.rnn.pack_padded_sequence(X, X_lenghts, batch_first=True)
        if hidden_state is None:
            outputs, state = self.lstm(X, self.initial_hidden)
        else:
            outputs, state = self.lstm(X, hidden_state)
        # outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs, batch_first=True)
        return self.output(outputs), state    

Dataset and data loader, again, works for batch size 1.

In [14]:
import torch.utils.data

start_idx = tag_map['<start>']
end_idx = tag_map['$']
class ShakespeareDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, x):
        return torch.tensor([start_idx] + [tag_map[i] for i in self.data[x]] + [end_idx])
    
    def __len__(self):
        return len(self.data)

padding_idx = tag_map['<pad>']
def collate_fn(x):
    x = sorted(x, key=lambda x: -x.size()[0])
    x_padded = torch.nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=padding_idx)
    return x_padded, torch.tensor([len(i) - 1 for i in x])

s = ShakespeareDataset([s for s in shakespeare if s])
shakespeare_loader = torch.utils.data.DataLoader(
    s,
    batch_size=1,
    num_workers=0,
    shuffle=True,
    pin_memory=True,
    collate_fn=collate_fn
)

Training with some generation of examples. The pbar use is pretty cool. We train on symbols of a sequence trying to predict the next symbol, thats why we do forward pass on [0, :-1] and run criterion on [0, 1:].

Generator temperature parameter helps us to smooth out the output probabilities a bit, so the samples sequences are more varied.

In [15]:
import tqdm

def train(model, optimizer, criterion, epochs=10, lr_decay=0.9, generator_temp=0.8):
    losses = []
    for epoch in range(epochs):
        losses.append([])
        pbar = tqdm.tqdm(enumerate(shakespeare_loader), total=len(shakespeare_loader))
        loss_avg = 0
        model.train()
        for i, (X, X_lengths) in pbar:
            X_cuda = X.to(device=device)
            preds, _ = model(X_cuda[0, :-1].view(1, -1), X_lengths)
            loss = 0
            for pred, true in zip(preds, X_cuda[0, 1:]):
                loss += criterion(pred, true)
            loss_avg += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 500 == 0:
                losses[-1].append(loss_avg / 500)
                pbar.set_description('loss: {:.4f}'.format(loss_avg))
                loss_avg = 0
        with torch.no_grad():
            model.eval()
            for _ in range(10):
                p = []
                predicted = tag_map['<start>']
                state = None
                for i in range(1000):
                    vec = torch.tensor([predicted]).to(device=device)
                    len_one = torch.tensor([1])
                    probs, state = model.forward(vec.unsqueeze(0), len_one, state)
                    predicted = torch.multinomial(probs.data.view(-1).div(generator_temp).exp(), 1)[0]
                    if predicted == end_idx:
                        break
                    p.append(tags[predicted])
                print(''.join(p).encode('ascii'))
        for group in optimizer.param_groups:
            group['lr'] = group['lr'] * 0.9
    return losses

The training can take quite a while, but there should be a trained model provided with the same result as all seeds are fixed.

In [16]:
model = ShakespeareRNN(num_layers=3, hidden_size=100, embedding_dim=100).to(device=device)
pretrained = True
if pretrained:
    model.load_state_dict(torch.load('model'))
else:
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    train(model, optimizer, criterion, epochs=10, lr_decay=0.9, generator_temp=0.8)
    train(model, optimizer, criterion, epochs=10, lr_decay=0.97, generator_temp=0.8)
    torch.save(model.state_dict(), 'model')

Alright, you can have some fun playing with the model as an oracle.

In [17]:
model.eval()
def predict(model, string):
    encoded = [tag_map['<start>']] + [tag_map[c] for c in string]
    probs, state = model.forward(torch.tensor([encoded]).to(device=device),
                                 torch.tensor([len(encoded)]))
    return probs, state

def oracle(model, sequence_init, generator_temp=0.8):
    probs, state = predict(model, sequence_init)
    p = [c for c in sequence_init]
    predicted = torch.multinomial(probs[0][-1].data.view(-1).div(generator_temp).exp(), 1)[0]
    p.append(tags[predicted])
    for i in range(1000):
        vec = torch.tensor([predicted]).to(device=device)
        len_one = torch.tensor([1])
        probs, state = model.forward(vec.unsqueeze(0), len_one, state)
        predicted = torch.multinomial(probs.data.view(-1).div(generator_temp).exp(), 1)[0]
        if predicted == end_idx:
            break
        p.append(tags[predicted])
    return ''.join(p).encode('ascii')

In [18]:
for i in range(5):
    print(oracle(model, 'Love ', 0.8))

b'Love is a pierced else avoid the storance.'
b'Love tod so.'
b'Love the very man.'
b'Love speak lord; they see what of the kind of uldess the very hearts are great fatal for the name solenculate.'
b"Love to come with been for all wonderous man for being of the cause; Let again of speak'st to counterform to him."


Now, we shall combine the Huffman code and probabilities generated by our model. Note that we don't need to encode the sequence end symbol '$'. You can encode him as well, I suppose, if you want to encode a stream of messages I guess. Its easy to include him in the code, but here I'll show how to exclude him. <start> and <pad> were never even outputed by the model.

First lets see what we are working with.

In [19]:
code_tags = tags[0:end_idx] + tags[end_idx + 1:-2]
def next_char_code(model, sequence):
    probs, _ = predict(model, sequence)
    probs = probs.cpu().detach()[0].numpy()
    p = probs[-1]
    p = np.hstack([p[0:end_idx], p[end_idx + 1:]])
    p = np.exp(p) / sum(np.exp(p))
    return sorted(huffman(zip(code_tags, p)).items(), key=lambda x: -x[1][0])

In [20]:
next_char_code(model, '')

[('W', (0.14264749, '101')),
 ('I', (0.11243192, '011')),
 ('A', (0.10398333, '001')),
 ('T', (0.09462893, '000')),
 ('H', (0.08030467, '1110')),
 ('N', (0.07071935, '1100')),
 ('M', (0.053659864, '0101')),
 ('S', (0.052735902, '0100')),
 ('O', (0.04119472, '11011')),
 ('Y', (0.03658414, '11010')),
 ('B', (0.03173486, '10010')),
 ('G', (0.026395047, '10000')),
 ('C', (0.025057951, '111111')),
 ('F', (0.02332487, '111110')),
 ('P', (0.021805987, '111101')),
 ('D', (0.02079137, '111100')),
 ('L', (0.017386848, '100111')),
 ("'", (0.013624064, '100010')),
 ('R', (0.0071174665, '1000111')),
 ('E', (0.007011211, '1000110')),
 ('V', (0.0044000708, '10011010')),
 ('U', (0.00378024, '10011001')),
 ('K', (0.0028912532, '100110111')),
 ('J', (0.0011139028, '1001101100')),
 ('e', (0.0007961873, '1001100001')),
 ('o', (0.0007092589, '1001100000')),
 ('Q', (0.00063566153, '10011011010')),
 ('a', (0.0004057145, '10011000100')),
 ('.', (0.00028089195, '100110001111')),
 ('v', (0.0001789701, '10011011

That makes some sense, capital letters are more likely to begin a sequence. Lets try something harder.

In [21]:
next_char_code(model, 'Begone you foul scoun')

[('d', (0.43711847, '0')),
 ('t', (0.38024732, '11')),
 ('c', (0.04595531, '1010')),
 ('s', (0.03772517, '1000')),
 (' ', (0.028689615, '10111')),
 ("'", (0.01926543, '10010')),
 ('k', (0.011997869, '100111')),
 (',', (0.006857081, '1011001')),
 ('f', (0.0053545586, '1001101')),
 ('?', (0.0034888429, '10110101')),
 ('n', (0.0034238726, '10110100')),
 ('i', (0.0034004694, '10110001')),
 ('y', (0.003317221, '10110000')),
 ('.', (0.002400817, '10011000')),
 ('!', (0.002331932, '101101111')),
 ('e', (0.0019245385, '101101101')),
 ('j', (0.0014458073, '100110011')),
 ('-', (0.0011579316, '1011011101')),
 ('l', (0.0009663824, '1011011100')),
 (';', (0.00049878354, '10110110011')),
 ('g', (0.00035039728, '10110110000')),
 (':', (0.0003460674, '10011001011')),
 ('a', (0.00031722392, '10011001001')),
 ('u', (0.00022955127, '10011001000')),
 ('o', (0.00022947072, '101101100101')),
 ('m', (0.0002288651, '101101100100')),
 ('r', (0.00019759138, '101101100010')),
 ('w', (0.00018167713, '10011001010

Excellent! What else?

In [22]:
for i in range(5):
    print(oracle(model, 'Give me a kiss, my '))
next_char_code(model, 'Give me a kiss, my ')

b'Give me a kiss, my lord, or no. Let and order and true.'
b'Give me a kiss, my lord.'
b'Give me a kiss, my lord.'
b"Give me a kiss, my lord of before thee more as we must be burn'd for it?"
b'Give me a kiss, my lord.'


[('l', (0.82327163, '1')),
 ('g', (0.022836115, '0101')),
 ('d', (0.018473243, '0011')),
 ('f', (0.017375054, '0001')),
 ('n', (0.016868433, '0000')),
 ('s', (0.01472008, '01110')),
 ('m', (0.014674191, '01101')),
 ('r', (0.012373771, '01001')),
 ('L', (0.010317123, '01000')),
 ('p', (0.008343307, '011111')),
 ('b', (0.008193638, '011110')),
 ('h', (0.007848793, '011001')),
 ('w', (0.0046810685, '001010')),
 ('t', (0.004225268, '001000')),
 ('c', (0.0033100492, '0110001')),
 ('v', (0.0026347137, '0010111')),
 ('o', (0.0024251158, '0010110')),
 ('a', (0.0016070531, '01100001')),
 ('e', (0.001193854, '00100111')),
 ('k', (0.0010430771, '00100101')),
 ('q', (0.00056913967, '001001101')),
 ('C', (0.0005485768, '001001100')),
 ('i', (0.0003941601, '0110000010')),
 ("'", (0.0003294751, '0110000001')),
 ('u', (0.00027932762, '0010010011')),
 (' ', (0.00025713586, '0010010010')),
 ('G', (0.00020162125, '01100000110')),
 ('R', (0.0001413505, '01100000000')),
 ('H', (0.0001239568, '00100100010')

Notice how confident he is here that the next character is 'l'. Lets just sanity check that for a moment.

In [23]:
import re
counter = Counter()
count = 0
reg = re.compile(' my \S*')
for s in shakespeare:
    m = reg.findall(s)
    count += len(m)
    counter.update(m)
print(count)
sorted(counter.items(), key=lambda x: -x[1])

10465


[(' my lord,', 460),
 (' my lord.', 286),
 (' my good', 201),
 (' my lord', 144),
 (' my heart', 126),
 (' my lord;', 115),
 (' my lord?', 103),
 (' my father', 101),
 (' my life,', 75),
 (" my father's", 64),
 (' my life', 62),
 (' my Lord', 61),
 (' my love', 60),
 (' my troth,', 60),
 (' my soul', 59),
 (' my dear', 58),
 (' my lord:', 57),
 (' my master', 56),
 (' my brother', 56),
 (' my noble', 54),
 (' my poor', 51),
 (' my heart,', 51),
 (' my soul,', 51),
 (' my lord!', 48),
 (' my daughter', 47),
 (' my liege,', 44),
 (' my lords,', 44),
 (' my sweet', 41),
 (' my heart.', 36),
 (' my father,', 35),
 (' my lady', 35),
 (' my son', 34),
 (' my son,', 34),
 (' my life.', 34),
 (' my head', 33),
 (' my tongue', 33),
 (' my name', 31),
 (' my cousin', 31),
 (' my mother', 30),
 (' my gracious', 30),
 (' my wife', 28),
 (' my love,', 27),
 (' my old', 26),
 (' my part,', 26),
 (' my most', 25),
 (' my mind', 24),
 (' my house', 24),
 (' my young', 24),
 (' my true', 24),
 (' my sw

Hopefully that explains the code above a little. The model is overconfident somewhat, but that is a common issue with neural networks: they do not learn distributions as much as find the most likely outcome. Which is good when you need to have a decision, but could cause problems if you expect an actual table of probabilities (as do we for encoding).

Lets try to encode stuff. But first some disclaimers. Encoded/compressed stuff usually has two components, the coding table (i.e. model) and the encoded content. The goal of efficient code is to minimize the size of coding table + size of encoded content. That makes our approach problematic since we have a huge model, so I will ignore that componet. Also, when encoding a shorter sequence, if some characters don't appear, they don't even need to have a code assigned, our model here however always predicts for all characters (which could be hacked around, but it does not match a model trained only on a subset of characters). So in short, I will compare our model to a 'fake standard' encoding method.

In [24]:
import sys
tags_total = sum(count for tag, count in tag_counts.items() if tag != '$')
symbol_freqs = []
for tag in tags:
    if tag != '$':
        symbol_freqs.append((tag, tag_counts[tag] / tags_total))
standard_code = huffman(symbol_freqs)
inverse_standard_code = {sf[1][1]:sf[0] for sf in standard_code.items()}
def standard_encode(sequence):
    encoded = []
    for s in sequence:
        encoded.append(standard_code[s][1])
    return ''.join(encoded)
def standard_decode(encoded):
    start = 0
    symbols = []
    for end in range(len(encoded) + 1):
        decoded = inverse_standard_code.get(encoded[start:end], False)
        if decoded:
            symbols.append(decoded)
            start = end
    return ''.join(symbols)

Lets just test it out.

In [25]:
encoded = standard_encode('Ye olde test sequence.')
print(len(encoded), encoded)
print(standard_decode(encoded))

101 11001110111000111100110111100000001111010000010010101110100000110001010110110000000111000100001011001
Ye olde test sequence.


Lets make the same for using the trained model.

In [26]:
def make_code(p):
    p = np.hstack([p[:end_idx], p[end_idx + 1:]])
    p = np.exp(p) / sum(np.exp(p))
    return huffman(zip(code_tags, p))

def shakespeare_encode(model, sequence):
    probs, _ = predict(model, sequence)
    probs = probs.cpu().detach()[0].numpy()
    encoded = []
    for s, p in zip(sequence, probs):
        code = make_code(p)
        encoded.append(code[s][1])
    return ''.join(encoded)

def shakespeare_decode(model, encoded):
    last_tag = start_idx
    start = 0
    state = None
    symbols = []
    while start < len(encoded):
        vec = torch.tensor([last_tag]).to(device=device)
        len_one = torch.tensor([1])
        probs, state = model.forward(vec.unsqueeze(0), len_one, state)
        p = probs.cpu().detach()[0].numpy()[-1]
        code = make_code(p)
        inverse_code = {sf[1][1]:sf[0] for sf in code.items()}
        sanity = 0
        for end in range(start, len(encoded) + 1):
            decoded = inverse_code.get(encoded[start:end], False)
            if decoded:
                symbols.append(decoded)
                start = end
                last_tag = tag_map[decoded]
                sanity = 1
                break
        if not sanity:
            print('Something went terribly wrong', start, encoded[start:], code)
            break
    return ''.join(symbols)

In [27]:
encoded = shakespeare_encode(model, 'Ye olde test sequence.')
print(len(encoded), encoded)
print(shakespeare_decode(model, encoded))

83 11010010000011010000111011111000000011001101001111111111100101100111011001100110110
Ye olde test sequence.


Excellent! It works! Lets try it out a bit now.

In [28]:
def test(sequence):
    sequence = ''.join(s for s in sequence if s in tag_map)
    print(sequence)
    encoded = standard_encode(sequence)
    print('Standard   ', len(encoded), encoded[:100])
    assert(sequence == standard_decode(encoded))
    encoded = shakespeare_encode(model, sequence)
    print('Shakespeare', len(encoded), encoded[:100])
    assert(sequence == shakespeare_decode(model, encoded))

In [29]:
test('I love the smell of napalm in the morning.')

I love the smell of napalm in the morning.
Standard    182 0010101111101111001011011100011110100101000111010000100000101111011111110011000111110011011111000110
Shakespeare 121 0111111010011111111011100100000101111111111110101101010111111011011001001101100001010000000011110110


In [30]:
test('To be or not to be, that is the question.')

To be or not to be, that is the question.
Standard    177 1100010010011111100110000111100111011111001110011010111101010011111100110000110000111101001010111101
Shakespeare 85 0000011111111001011010010011011100010101011011110111101110001111111100000111011111101


As you might expect, the more 'Shakespearian' a sequence looks the better it will be encoded. Additionally, the language model has can infer partial words pretty well. It should have an unfair advantage to the standard model as long as the language is similar to english as it uses context to encode better.

Lets try a foreign example. This is from the first paragraph on William Shakespeare on German language Wikipedia, modified so it only has mapped characters.

In [31]:
test('William Shakespeare war ein englischer Dramatiker, Lyriker und Schauspieler.'
     ' Seine Komodien und Tragodien gehoren zu den bedeutendsten Buhnenstucken der'
     ' Weltliteratur und sind die am haufigsten aufgefuhrten und verfilmten. Das '
     'uberlieferte Gesamtwerk umfasst Dramen, epische Versdichtungen sowie Sonette.')

William Shakespeare war ein englischer Dramatiker, Lyriker und Schauspieler. Seine Komodien und Tragodien gehoren zu den bedeutendsten Buhnenstucken der Weltliteratur und sind die am haufigsten aufgefuhrten und verfilmten. Das uberlieferte Gesamtwerk umfasst Dramen, epische Versdichtungen sowie Sonette.
Standard    1403 0010100011010101111011111010011100100111001010011010101110110101000010011000110000111110110001111011
Shakespeare 1724 1010011101111110111111000010101110110110101010010001100000110101001111111010101110001110101000101110


And here it has worse, as the distribution is sufficiently different from the one the model was trained on.

And thats it. Feel free to play around with stuff above and I hope you had fun.